# Lab in Python {#sec-map-raster-Python .unnumbered}

In this session, we will further explore the world of geographic data visualization by building upon our understanding of both raster data and choropleths. Raster data, consisting of gridded cells, allows us to represent continuous geographic phenomena such as temperature, elevation, or satellite imagery. Choropleths, on the other hand, are an effective way to visualize spatial patterns through the use of color-coded regions, making them invaluable for displaying discrete data like population density or election results. By combining these techniques, we will gain a comprehensive toolkit for conveying complex geographical information in a visually compelling manner.

## Importing Modules


In [ ]:
# Raster modules for python
import rasterio
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.mask import mask
from rasterio.plot import show
from rasterio import features

from shapely.geometry import mapping

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
import matplotlib.colorbar as colorbar
from matplotlib.colors import ListedColormap, BoundaryNorm

import geopandas as gpd

## Terrain data

### **Import raster data**

Raster **terrain** data consists of gridded elevation values that represent the topography of a geographic area. You can download this from the [relevant github folder](https://github.com/pietrostefani/gds/tree/main/data/Lebanon). A good place to download elevation data is [Earth Explorer](https://earthexplorer.usgs.gov/). This [video](https://www.youtube.com/watch?v=NQg0g9ObhXE) takes you through the download process if you want to try this out yourself.

We first import a raster file for elevation.


In [ ]:
# Load the raster data
elevation = rasterio.open("data/Lebanon/LBN_elevation_w_bathymetry.tif")

Plot it.


In [ ]:
plt.figure(figsize=(10, 8))
plt.imshow(elevation.read(1), cmap='viridis')
plt.colorbar(label='Elevation')
plt.title('Elevation with Bathymetry')
plt.show()

This information is typically accessed and updated via the .profile.


In [ ]:
print(elevation.profile)

Have a look at the CRS.


In [ ]:
# Check the CRS of the raster
crs = elevation.crs
print(crs)

### **Import the Lebanon shapefile**

Import the Lebanon shapefile, plot it, and verify its Coordinate Reference System (CRS). Is it the same as the raster's CRS?


In [ ]:
# Load the shapefile data
Lebanon_adm1 = gpd.read_file("data/Lebanon/LBN_adm1.shp")

# Plot the geometry
Lebanon_adm1.plot(edgecolor='grey', facecolor='none')
plt.title('Lebanon Administrative Boundaries')
plt.show()

In [ ]:
# Check the CRS of the shapefile
crs = Lebanon_adm1.crs
print(crs)

### **Reproject the Raster**


In [ ]:
# Define the desired destination CRS
dst_crs = "EPSG:22770"  # For example, WGS84

# Calculate the transform matrix, width, and height for the output raster
dst_transform, width, height = calculate_default_transform(
    elevation.crs,  # source CRS from the raster
    dst_crs,        # destination CRS
    elevation.width,  # column count
    elevation.height, # row count
    *elevation.bounds  # outer boundaries (left, bottom, right, top)
)

# Print the source and destination transforms
print("Source Transform:\n", elevation.transform, '\n')
print("Destination Transform:\n", dst_transform)

# Define the metadata for the output raster
dst_meta = elevation.meta.copy()
dst_meta.update({
    'crs': dst_crs,
    'transform': dst_transform,
    'width': width,
    'height': height
})

# Reproject and write the output raster
with rasterio.open("data/Lebanon/reprojected_elevation.tif", "w", **dst_meta) as dst:
    for i in range(1, elevation.count + 1):
        reproject(
            source=rasterio.band(elevation, i),
            destination=rasterio.band(dst, i),
            src_transform=elevation.transform,
            src_crs=elevation.crs,
            dst_transform=dst_transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )

### **Cropping and Masking**

**Cropping**:

-   Purpose: Cropping a raster involves changing the extent of the raster dataset by specifying a new bounding box or geographic area of interest. The result is a new raster that covers only the specified region.

-   Typical Use: Cropping is commonly used when you want to reduce the size of a raster dataset to focus on a smaller geographic area of interest while retaining all the original data values within that area.

**Masking**:

-   Purpose: Applying a binary mask to the dataset. The mask is typically a separate raster or polygon layer where certain areas are designated as "masked" (1) or "unmasked" (0).

-   Typical Use: Masking is used when you want to extract or isolate specific areas or features within a raster dataset. For example, you might use a mask to extract land cover information within the boundaries of a protected national park.

In many cases, these cropping and masking are executed one after the other because it is computationally easier to crop when dealing with large datasets, and then masking.


In [ ]:
elevation_22770 = rasterio.open("data/Lebanon/reprojected_elevation.tif")

# Use unary_union method to combine the geometries
lebanon_union = Lebanon_adm1.geometry.unary_union

# Crop the elevation data to the extent of Lebanon
elevation_lebanon, elevation_lebanon_transform = mask(elevation_22770, [mapping(lebanon_union)], crop=True)

### **Plot elevation**


In [ ]:
# Assuming elevation_lebanon contains the cropped elevation data and Lebanon_adm1 is the GeoDataFrame
fig, ax = plt.subplots(figsize=(10, 10))
# Plot the elevation data
show(elevation_lebanon, transform=elevation_lebanon_transform, ax=ax, cmap='terrain')
# Plot the Lebanon boundaries on top, with no fill color
Lebanon_adm1.boundary.plot(ax=ax, edgecolor='black')

plt.show()

Let's improve this a bit. Remember that there is a lot we can do with [Cmap](https://www.analyticsvidhya.com/blog/2020/09/colormaps-matplotlib/#:~:text=Colormaps%20or%20Cmap%20in%20python%20colormaps%20is%20a%20very%20useful,custom%20ones%20using%20python%20colormaps).


In [ ]:
# Define the reversed 6 shades of orange
orange_shades_reversed = ['#ef3b2c', '#fb6a4a', '#fc9272', '#fcbba1', '#fee0d2', '#fff5eb']

# Define the breaks
boundaries = [-100, 0, 700, 1200, 1800, 3300]

# Define the color map and normalization
cmap = ListedColormap(orange_shades_reversed)
norm = BoundaryNorm(boundaries=boundaries, ncolors=len(orange_shades_reversed))

fig, ax = plt.subplots(figsize=(10, 10))

# Plot the elevation data with the custom color map
im = show(elevation_lebanon, transform=elevation_lebanon_transform, ax=ax, cmap=cmap, norm=norm)

# Plot the Lebanon boundaries on top, with no fill color
Lebanon_adm1.boundary.plot(ax=ax, edgecolor='black')

# Remove the axes
ax.axis('off')

# Manually create a legend
legend_labels = ['< 0 m', '0 - 700 m', '700 - 1200 m', '1200 - 1800 m', '1800 - 3300 m', '> 3300 m']
legend_patches = [Patch(color=orange_shades_reversed[i], label=legend_labels[i]) for i in range(len(orange_shades_reversed))]

# Add the legend to the right of the plot
ax.legend(handles=legend_patches, loc='center left', bbox_to_anchor=(1, 0.5), title='Elevation (m)', frameon=False)


plt.show()

Questions to ask yourself about how you can improve these maps, going back to [geo-visualisation and choropleths](https://pietrostefani.github.io/gds/mapvector.html).

-   What are the logical breaks for elevation data?

-   Should the colours be changed to standard elevation pallettes?

### **Slope**

We are now going to calculate slopes

### **Flood risk area**

We can employ reclassification techniques to delineate flood risk areas. Specifically, we identify and classify areas where the elevation is under 10 meters above sea level as high-risk zones. This critical step in flood risk assessment helps communities and decision-makers pinpoint vulnerable areas, enabling them to implement effective mitigation strategies and disaster preparedness plans to safeguard against potential flooding events.

<!--
problem installing osgeo and richdem in 3.11 python

https://www.earthdatascience.org/tutorials/visualize-digital-elevation-model-contours-matplotlib/

https://www.earthdatascience.org/tutorials/get-slope-aspect-from-digital-elevation-model/
-->

### **Spatial join with vector data**

You might want to extract values from a raster data set, and then map them within a vector framework or extract them to analyse them statistically. If it therefore very useful to know how to extract:


In [ ]:
# Load some geo-localized survey data
households = gpd.read_file("data/Lebanon/random_survey_LBN.shp")

# Open the elevation raster file
with rasterio.open("data/Lebanon/LBN_elevation_w_bathymetry.tif") as src:
    # Reproject households coordinates to the CRS of the raster
    households = households.to_crs(src.crs)
    
    # Extract elevation values at the coordinates of the points
    housesales_elevation = [
        val[0] if val is not None else None
        for val in src.sample([(geom.x, geom.y) for geom in households.geometry])
    ]

# Attach elevation at each point to the original households GeoDataFrame
households['elevation'] = housesales_elevation

# Check out the data
print(households.head())

- *Handling CRS (Coordinate Reference System)*: The household data CRS is transformed to match the raster’s CRS before extracting elevation values.
- *Extracting Elevation*: Elevation values are extracted at each household location using rasterio’s sample method.
- *Attaching Elevation Data*: The elevation data is added as a new column to the households `GeoDataFrame`.
	
::: callout-important
Make sure all your data is in the same CRS, otherwise the `rasterio`’s sample will not work properly.
:::

## Night Lights

This section is a bit more advanced, there are hints along the way to make it simpler.

### **Download data**

::: {.callout-note title="Download the Data"}

We need to download some raster data. NOAA has made nighttime lights data available for 1992 to 2013. It is called the Version 4 DMSP-OLS Nighttime Lights Time Series. The files are cloud-free composites made using all the available archived DMSP-OLS smooth resolution data for calendar years. In cases where two satellites were collecting data - two composites were produced. The products are 30 arc-second grids, spanning -180 to 180 degrees longitude and -65 to 75 degrees latitude. We can download the [Average, Visible, Stable Lights, & Cloud Free Coverages for 1992 and 2013](https://www.ngdc.noaa.gov/eog/data/web_data/v4composites/) and put them in the `data/Kenya_Tanzania` folder.

::: 

::: callout-important
You can also download the data [here](https://theuniversityofliverpool-my.sharepoint.com/:u:/g/personal/pietrost_liverpool_ac_uk/EX85foAh7ZBBtSZ64pxbFmsBStf8UX2uaLXE5MBYJt6PeQ?e=LMYu7d). You will need to be logged into your UoL account.

The downloaded files are going to be in a **"TAR" format**. A TAR file is an archive created by tar, a Unix-based utility used to package files together for backup or distribution purposes. It contains multiple files stored in an uncompressed format along with metadata about the archive. Tars files are also used to reduce files' size. TAR archives compressed with GNU Zip compression may become GZ, .TAR.GZ, or .TGZ files. We need to decompress them before using them.
:::

Before you move forward **download the data for 1992 and 2013.** It is also good practice to create a scratch folder where you do all your unzipping.


In [ ]:
def load_raster(filename):
    """Load a raster file and return the data as a numpy array."""
    with rasterio.open(filename) as src:
        data = src.read(1)  # Read the first band
        profile = src.profile  # Save the profile for later use
    return data, profile

def stack_rasters(*rasters):
    """Stack multiple raster arrays into a single 3D numpy array."""
    return np.stack(rasters, axis=-1)

def plot_raster(data, title='Raster Data'):
    """Plot a single raster array."""
    plt.figure(figsize=(10, 6))
    plt.imshow(data, cmap='viridis', interpolation='none')
    plt.colorbar()
    plt.title(title)
    plt.show()

# File paths to your raster TIFF files
file1 = 'nighttime_lights_1.tif'
file2 = 'nighttime_lights_2.tif'

# Load the rasters
data1, profile1 = load_raster(file1)
data2, profile2 = load_raster(file2)

# Stack the rasters
stacked_data = stack_rasters(data1, data2)

We can load and plot the nighttime lights data. When working with many rasters of the same origin, it is usually faster to stack them together for faster processing.


In [ ]:
# Plot the first raster
plot_raster(data1, title='Nighttime Lights - File 1')

# Plot the second raster
plot_raster(data2, title='Nighttime Lights - File 2')

# Plot the stacked rasters (showing all bands together)
plt.figure(figsize=(10, 6))
plt.imshow(np.mean(stacked_data, axis=-1), cmap='viridis', interpolation='none')
plt.colorbar()
plt.title('Stacked Nighttime Lights Data (Mean of All Bands)')
plt.show()

### **Country shapefiles**

### **Zonal statistics**

### **Merge Shapefiles**

### **Visualize**


## Resources

-   [Python Open Source Spatial Programming & Remote Sensing](https://pygis.io/docs/e_interpolation.html)

- [Remote Sensing with Python](https://worldbank.github.io/OpenNightLights/welcome.html)

- [Black Marble in Python](https://blogs.worldbank.org/en/opendata/illuminating-insights-harnessing-nasas-black-marble-r-and-python-packages)